In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-managed-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/spark/spark_ml.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This notebook tutorial shows you Apache SparkML jobs with Dataproc and BigQuery. Through this notebook, you can learn a common use case in the machine learning pipeline: ingestion, data cleaning, feature engineering, modeling, and evaluation.

### Dataset

The two datasets, [NYC TLC (Taxi and Limousine Commission) Trips](https://console.cloud.google.com/marketplace/product/city-of-new-york/nyc-tlc-trips) dataset and [NYC Citi Bike Trips](https://console.cloud.google.com/marketplace/product/city-of-new-york/nyc-citi-bike) dataset, are available in [BigQuery Public Datasets](https://cloud.google.com/bigquery/public-data), and provides free querying of up to 1TB of data each month. It contains trips data for each Taxi and Citi Bike, the public bicycle sharing system serving New York City.

### Objective

This notebook tutorial runs an Apache SparkML job that fetches data from the BigQuery dataset, performs exploratory data analysis, cleans data, executes feature engineering, trains the model, evaluates the model, debriefs for the result and saves the model to a Cloud Storage bucket.

This notebook tutorial performs the following steps:

- Setting up a Google Cloud project and Dataproc cluster.
- Creating a Cloud Storage bucket and a BigQuery dataset.
- Configuring the spark-bigquery-connector.
- Ingesting data from BigQuery into a Spark DataFrame.
- Performing Exploratory Data Analysis (EDA).
- Visualizing data with samples.
- Cleaning data.
- Selecting features.
- Training the model.
- Debriefing the result
- Saving the model to a Cloud Storage bucket.
- Deleting the resources created for this notebook tutorial.

### Costs 

This tutorial uses billable components of Google Cloud:

* [Vertex AI](https://cloud.google.com/vertex-ai/pricing)
* [Cloud Storage](https://cloud.google.com/storage/pricing)
* [Dataproc](https://cloud.google.com/dataproc/pricing)
* [BigQuery](https://cloud.google.com/bigquery/pricing)

You can use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

### Set up your Google Cloud project:

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you create an account, you receive a $300 credit towards to your compute and storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the BigQuery API, Notebooks API, Vertex AI API, and Dataproc API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,notebooks.googleapis.com,aiplatform.googleapis.com,dataproc&_ga=2.209429842.1903825585.1657549521-326108178.1655322249)

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and inserts the value of Python variables prefixed with `$` into the commands.

### Installation

Install the following packages to run this notebook.

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

USER_FLAG = ""
# Google Cloud Notebook requires dependencies to be installed with '--user'
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
if os.getenv("IS_TESTING"):
    """
    The testing suite does not currently support testing on Dataproc clusters,
    so the testing environment is setup to replicate Dataproc via the following steps.
    """
    JAVA_VER = "8u332-b09"
    JAVA_FOLDER = "/tmp/java"
    FILE_NAME = f"openlogic-openjdk-{JAVA_VER}-linux-x64"
    TAR_FILE = f"{JAVA_FOLDER}/{FILE_NAME}.tar.gz"
    DOWNLOAD_LINK = f"https://builds.openlogic.com/downloadJDK/openlogic-openjdk/{JAVA_VER}/openlogic-openjdk-{JAVA_VER}-linux-x64.tar.gz"
    PYSPARK_VER = "3.1.3"

    # Download Open JDK 8. Spark requires Java to execute.
    ! rm -rf $JAVA_FOLDER
    ! mkdir $JAVA_FOLDER
    ! wget -P $JAVA_FOLDER $DOWNLOAD_LINK
    os.environ["JAVA_HOME"] = f"{JAVA_FOLDER}/{FILE_NAME}"
    ! tar -zxf $TAR_FILE -C $JAVA_FOLDER
    ! echo $JAVA_HOME

    # Pin the Spark version to match that the Dataproc 2.0 cluster.
    ! pip install {USER_FLAG} pyspark==$PYSPARK_VER -q

    # Install GeoPandas.
    ! pip install {USER_FLAG} geopandas -q

    # Install PyArrow.
    ! pip install {USER_FLAG} pyarrow -q

    # Install rtree.
    ! pip install {USER_FLAG} rtree -q

    # Install seaborn.
    ! pip install {USER_FLAG} seaborn -q

### Create a Dataproc cluster

The Spark job executed in this notebook tutorial is compute intensive. Since the job can take a significant amount time to complete in a standard notebook environment, this notebook tutorial runs on a Dataproc cluster that is created with the Dataproc Component Gateway and Jupyter component installed on the cluster.

**Existing Dataproc with Jupyter cluster?**: If you have a running Dataproc cluster that has the [Component Gateway and Jupyter component installed on the cluster](https://cloud.google.com/dataproc/docs/concepts/components/jupyter#gcloud-command)), you can use it in this tutorial. If you plan to use it, skip this step, and go to `Switch your kernel`.

In [ ]:
if not os.getenv("IS_TESTING"):
    CLUSTER_NAME = "[your-cluster]"  # @param {type: "string"}
    CLUSTER_REGION = "[your-region]"  # @param {type: "string"}

    if CLUSTER_REGION == "[your-region]":
        CLUSTER_REGION = "us-central1"

    print(f"CLUSTER_NAME: {CLUSTER_NAME}")
    print(f"CLUSTER_REGION: {CLUSTER_REGION}")

Your `CLUSTER_NAME` must be **unique within your Google Cloud project**. It must start with a lowercase letter, followed by up to 51 lowercase letters, numbers, and hyphens, and cannot end with a hyphen.

In [ ]:
if not os.getenv("IS_TESTING"):
    !gcloud dataproc clusters create $CLUSTER_NAME \
        --region=$CLUSTER_REGION \
        --enable-component-gateway \
        --image-version=2.0 \
        --optional-components=JUPYTER

#### Switch your kernel

Your notebook kernel is listed at the top of the notebook page. Your notebook should run on the Python 3 kernel running on your Dataproc cluster.

Select **Kernel > Change Kernel** from the top menu, then select `Python 3 on CLUSTER_NAME: Dataproc cluster in REGION (Remote)`.

### Set your project ID

Run the following cell to get your project ID.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

If the previous command has no output, copy your project ID from the project selector in the [Google Cloud console](https://console.cloud.google.com/). Insert the ID in the `[your-project-id]` placeholder, then run the following command.

In [ ]:
if not PROJECT_ID or PROJECT_ID == "":
    PROJECT_ID = "[your-project-id]"  # @param {type: "string"}

In [ ]:
! gcloud config set project $PROJECT_ID -q

### Create a Cloud Storage bucket

The machine learning model created in this tutorial is stored in Cloud Storage bucket.

#### Region

Before creating a Cloud Storage bucket, re-define the `REGION` variable (when you changed the notebook kernel earlier, previously set variables were deleted).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

# If you do not specify a region, it will be "us-central1".
if not REGION or REGION == "" or REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

To avoid name collisions, you can create a UUID for the current notebook session, then append the UUID to the name of resources such as the Cloud Storage bucket and BigQuery dataset that you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

Replace the `[your-bucket-name]` placeholder with the name of your Cloud Storage bucket. The name must be unique across all Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}/"

# If you do not specify a bucket name, it will be created based on your project ID and the UUID.
if not BUCKET_NAME or BUCKET_NAME == "" or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = f"{PROJECT_ID}{UUID}"
    BUCKET_URI = f"gs://{BUCKET_NAME}/"

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Confirm your access to the Cloud Storage bucket by displaying the bucket's metadata:

In [ ]:
! gsutil ls -L -b $BUCKET_URI

### Create a Bigquery Dataset

In order to use a query from BigQuery dataset, creating a temporary table is necessary to save the result, of which Spark will read the results from.

In [ ]:
if not os.getenv("IS_TESTING"):
    DATASET_NAME = "[your-dataset-name]"  # @param {type:"string"}

    if (
        DATASET_NAME == ""
        or DATASET_NAME is None
        or DATASET_NAME == "[your-dataset-name]"
    ):
        DATASET_NAME = f"{PROJECT_ID}{UUID}"
else:
    DATASET_NAME = f"python_docs_samples_tests_{UUID}"

In [ ]:
! bq mk $DATASET_NAME

## Tutorial

### Import required libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from geopandas import gpd
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
# A Spark Session is how you interact with Spark SQL to create Dataframes
from pyspark.sql import SparkSession
# PySpark functions
from pyspark.sql.functions import (col, floor, pandas_udf, to_timestamp, udf,
                                   unix_timestamp)
# These allow us to create a schema for our data
from pyspark.sql.types import BooleanType, DoubleType
from shapely.geometry import Point

### Initialize the SparkSession

To use Apache Spark with BigQuery, you must include the [spark-bigquery-connector](https://github.com/GoogleCloudDataproc/spark-bigquery-connector) when you initialize the SparkSession.

In [ ]:
VER = "0.26.0"
FILE_NAME = f"spark-bigquery-with-dependencies_2.12-{VER}.jar"

if os.getenv("IS_TESTING"):
    connector = f"https://github.com/GoogleCloudDataproc/spark-bigquery-connector/releases/download/{VER}/{FILE_NAME}"
else:
    connector = f"gs://spark-lib/bigquery/{FILE_NAME}"


"""
Initialize the "SparkSession" with the following config. 
"viewsEnabled" and "materializationDataset" are required to enable to query data in spark-bigquery-connector.
For more information, check https://github.com/GoogleCloudDataproc/spark-bigquery-connector#reading-data-from-a-bigquery-query.
"""
spark = (
    SparkSession.builder.appName("spark-bigquery-polyglot-language-demo")
    .config("spark.jars", connector)
    .config("viewsEnabled", "true")
    .config("materializationDataset", f"{DATASET_NAME}")
    .getOrCreate()
)

### Fetch data from BigQuery

In this tutorial, NYC Taxi dataset is in 2016 and NYC Citi Bike dataset is in between 2013 and 2018. Since their time range are different, use SQL query to match both datasets.

In [ ]:
# Load NYC_taxi in Github Activity Public Dataset from BigQuery.
taxi_df = (
    spark.read.format("bigquery")
    .option("table", "bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2016")
    .load()
)

bike_sql = """
    SELECT * 
    FROM bigquery-public-data.new_york_citibike.citibike_trips 
    WHERE EXTRACT(YEAR FROM starttime)=2016 and EXTRACT(YEAR FROM stoptime)=2016
"""

# Load NYC_Citibike in Github Acitivity Public dataset from BQ.
bike_df = spark.read.format("bigquery").load(bike_sql)

"""
Although this is not a standard practice in a production environment, only 10% of the data for each dataset will be used in this project since it consumes too much time and computing resources. You can try different values for yourself but the runtime of the notebook linearly scales with the size of data.
"""
taxi_df = taxi_df.sample(0.1)
bike_df = bike_df.sample(0.1)

if os.getenv("IS_TESTING"):
    taxi_df = taxi_df.sample(0.0001)
    bike_df = bike_df.sample(0.0001)

### Perform Exploratory Data Analysis (EDA)

When you get started with a new problem, the first step is to gain an understanding of what the dataset contains. EDA is used to derive insights from the data. Data scientists and analysts try to find different patterns, relations, and anomalies in the data using some statistical graphs and other visualization techniques. It allows analysts to understand the data better before making any assumptions.

Check the data types for Taxi dataset first.

In [ ]:
taxi_df.printSchema()

Filter out unnecessary columns and check null counts of the fields.

In [ ]:
taxi_df = taxi_df.select(
    "pickup_datetime",
    "dropoff_datetime",
    "trip_distance",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "fare_amount",
)

taxi_df.describe().show()

From this summary, you are able to know a lot of information.
  * There are over 13 millions of trip history for Yellow Taxi in 2016. but it has some abnormal values such as null and negative values in it. For instance, non-null value of longitudes and latitudes are only about the half of the total rows.
  * `pickup_datetime` and `dropoff_datetime` are string format when it was converted from the BigQuery dataset. To use it effectively, it needs to be re-formatted.
  * In 2016, the exact latitude and longitude were used for the pickup and the dropoff locations. It raised a lot of [privacy concerns](https://agkn.wordpress.com/2014/09/15/riding-with-the-stars-passenger-privacy-in-the-nyc-taxicab-dataset/) and the dataset has been providing `pickup_location_id` and `dropoff_location_id` instead. This id is corresponded to the [NYC Taxi Zones](https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc), roughly based on NYC Department of City Planning’s Neighborhood Tabulation Areas (NTAs) and are meant to approximate neighborhoods.
  
Based on numbers shown above, filter out some unrealistic values.
  * Every row contains null values must be removed.
  * trip_distance must be in between 0 and 20 (miles) since the length of Manhattan is about 13.4 miles long.
  * pickup_latitude and dropoff_latitude must be in between 40 and 41
  * pickup_longitude and dropoff_longitude must be in between -75 to -73
  * fare_amount must be in between 0 and 500 (dollars) since it does not include taxes and tips.

In [ ]:
taxi_df = taxi_df.where(
    (col("trip_distance") > 0)
    & (col("trip_distance") < 20)
    & (col("fare_amount") > 0)
    & (col("fare_amount") < 500)
    & (col("pickup_latitude") > 40)
    & (col("pickup_latitude") < 41)
    & (col("dropoff_latitude") > 40)
    & (col("dropoff_latitude") < 41)
    & (col("pickup_longitude") > -75)
    & (col("pickup_longitude") < -73)
    & (col("dropoff_longitude") > -75)
    & (col("dropoff_longitude") < -73)
).dropna()

### Perform Feature Engineering

To convert the location to the NYC Taxi Zones, Pandas UDF, as known as Vectorized UDF feature is used. For more information, [check this link](https://www.databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html).

It converts the given coordinates into the Taxi Zone in NYC.

In [ ]:
# Load the GeoJSON format of NYC Taxi zones.
gdf_zone = gpd.read_file(
    "https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc?method=export&format=GeoJSON"
)

# Convert the type of "location_id" column.
gdf_zone["location_id"] = gdf_zone["location_id"].astype("long")

# Convert GeoPandas format to a dictionary.
hm = gdf_zone.to_dict("index")

# Create a set that contains location_id if the location is in Manhattan.
manhattan_set = {hm[i]["location_id"] for i in hm if hm[i]["borough"] == "Manhattan"}


@pandas_udf("long")
def preprocess_zone_id(lat: pd.Series, lon: pd.Series) -> pd.Series:
    """
    The preprocessing function takes latitudes and longitudes and returns the corresponding Taxi Zone id.
    Args:
        lat: The latitude of the position.
        lon: The longitude of the position.
    Returns:
        The Taxi Zone id.
    """
    point_var = [Point(xy) for xy in zip(lon, lat)]
    gdf_points = gpd.GeoDataFrame(
        pd.DataFrame({"lat": lat, "lon": lon}), crs="epsg:4326", geometry=point_var
    )
    gdf_joined = gpd.sjoin(gdf_points, gdf_zone, how="left")
    return gdf_joined["location_id"]


@udf(returnType=BooleanType())
def is_in_manhattan(location_id):
    """
    The preprocessing function takes location_id and returns whether the given location_id is in the manhattan_set.
    Args:
        location_id: A number from 0 to 263 that represents NYC Taxi Zone.
    Returns:
        A Boolean value of whether the given location_id is in the manhattan_set.
    """
    return location_id in manhattan_set

In [ ]:
# Add "start_zone_id" and "end_zone_id" by applying the UDF "preprocess_zone_id".
taxi_df = taxi_df.withColumn(
    "start_zone_id",
    preprocess_zone_id(taxi_df["pickup_latitude"], taxi_df["pickup_longitude"]),
)
taxi_df = taxi_df.withColumn(
    "end_zone_id",
    preprocess_zone_id(taxi_df["dropoff_latitude"], taxi_df["dropoff_longitude"]),
)

# Add "is_start_manhattan" and "is_end_manhattan" by applying the UDF "is_in_manhattan".
taxi_df = taxi_df.withColumn(
    "is_start_manhattan", is_in_manhattan(col("start_zone_id"))
)
taxi_df = taxi_df.withColumn("is_end_manhattan", is_in_manhattan(col("end_zone_id")))

# Trips must done within the Manhattan and between different taxi zones.
taxi_df = taxi_df.where(
    (col("start_zone_id") != col("end_zone_id"))
    & (col("is_start_manhattan"))
    & (col("is_end_manhattan"))
)

Use of `to_timestamp()` function and `unix_timestamp()` function to convert `pickup_datetime` and `dropoff_datetime` as a Unix Timestamp type.

Unix time is a way of representing time as the number of seconds since `January 1st, 1970 at 00:00:00 UTC`. Compared to the Timestamp type, Unix time can be represented as an integer, making it easier to parse and use across different systems.

After you get `start_time` and `end_time` by converting the original `pickup_datetime` and `dropoff_datetime`, you are able to get more insteresting columns manipulating these two Timestamps.

In [ ]:
# Convert the type of datetime from a string to a Unix timestamp.
taxi_df = taxi_df.withColumn(
    "start_time", unix_timestamp(to_timestamp(col("pickup_datetime")))
)
taxi_df = taxi_df.withColumn(
    "end_time", unix_timestamp(to_timestamp(col("dropoff_datetime")))
)

# Calculate if this is a weekday
taxi_df = taxi_df.withColumn(
    "is_weekday",
    ((floor(col("start_time") / 86400) + 4) % 7 > 0)
    & ((floor(col("start_time") / 86400) + 4) % 7 < 6),
)

# Convert start_time to start_time_in_minute.
taxi_df = taxi_df.withColumn(
    "start_time_in_minute", floor((col("start_time") % 86400) / 60)
)

# Calculate trip_duration.
taxi_df = taxi_df.withColumn("trip_duration", col("end_time") - col("start_time"))

Prior to visualizing this modified Taxi dataset, do similar feature engineering for the Citi Bike dataset.

In [ ]:
bike_df.printSchema()
bike_df.describe().show()

From this summary, there is also interesting information from the dataset's summary.
  - There are over 10 million logs of trips for Citi Bike in 2016.
  - The current dataset has some abnormal values.
  - `starttime` and `stoptime` are string format. To use it effectively, it needs to be re-formatted.

In [ ]:
# Rename "tripduration" to "trip_duration".
bike_df = bike_df.withColumnRenamed("tripduration", "trip_duration")

# Convert the type of start/stop time from a string to a Unix timestamp.
bike_df = bike_df.withColumn(
    "starttime", unix_timestamp(to_timestamp(col("starttime")))
)
bike_df = bike_df.withColumn("stoptime", unix_timestamp(to_timestamp(col("stoptime"))))

# Check whether the starttime is a weekday or a weekend.
bike_df = bike_df.withColumn(
    "is_weekday",
    ((floor(col("starttime") / 86400) + 4) % 7 > 0)
    & ((floor(col("starttime") / 86400) + 4) % 7 < 6),
)

# Convert starttime to start_time_in_minute
bike_df = bike_df.withColumn(
    "start_time_in_minute", floor((col("starttime") % 86400) / 60)
)

# Add "start_zone_id" and "end_zone_id" by applying the UDF "preprocess_zone_id".
bike_df = bike_df.withColumn(
    "start_zone_id",
    preprocess_zone_id(
        bike_df["start_station_latitude"], bike_df["start_station_longitude"]
    ),
)
bike_df = bike_df.withColumn(
    "end_zone_id",
    preprocess_zone_id(
        bike_df["end_station_latitude"], bike_df["end_station_longitude"]
    ),
)

# Add "is_start_manhattan" and "is_end_manhattan" by applying the UDF "is_in_manhattan".
bike_df = bike_df.withColumn(
    "is_start_manhattan", is_in_manhattan(col("start_zone_id"))
)
bike_df = bike_df.withColumn("is_end_manhattan", is_in_manhattan(col("end_zone_id")))

# Filter out the bike_df by the following conditions
bike_df = bike_df.where(
    (
        col("start_zone_id") != col("end_zone_id")
    )  # Trips must done between different taxi zones.
    & (
        col("starttime") < col("stoptime")
    )  # The start time must be earlier than the end time.
    & (col("is_start_manhattan"))
    & (col("is_end_manhattan"))  # Trips must start and end within the Manhattan
)

# Filter out unnecessary columns.
bike_df = bike_df.drop(
    "start_station_id",
    "start_station_name",
    "end_station_id",
    "end_station_name",
    "bikeid",
    "birth_year",
    "gender",
    "customer_plan",
)

Assume that the Citi Bike users move through the Manhattan's roads, which are the perfectly perpendicular streets, the trip distance can be calculated by applying the [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry).

To Calulate the Manhattan distance, assume that you are in the **W 15th St and 9th Ave, where Google NYC is located,** and you want to go to **W 33rd St and 5th Ave, where the Empire State building is located**. In a full of city blocks, the fastest way you can get to the destination is to move east until you reached the 5th Avenue, and move north until you reached the W 33rd St. In this case, **W 16th St and 5th Ave** or **W 33rd St and 9th Ave** can be a hinge point. If we set the starting point as **S**, the ending point as **E**, and the hinge point as **H**, the formula of Manhattan distance is `distance(S, H) + distance(H, E)`.

However, in a real world, it requires more calculation to get the "real" Manhattan distance. First of all, unlike the Cartesian coordinate system. the actual distance in longitude varies with latitude. For example, 1 degree of longitude represents roughly 69 miles at equator, is the same as latitude, while it is about 49 miles at 45 degrees North or South. To calculate the exact distance in a real world using latitudes and longitudes, [Haversine distance](https://en.wikipedia.org/wiki/Haversine_formula) is one of the good choices.

Furthermore, the streets in Manhattan are inclined at about 29 degrees to the True north, so you should rotate the point S and E by 29 degrees counter-clockwise.

Therefore, the final formula should be `manhattan_distance = haversine_distance(S', H') + haversine_distance(H', E')` where `S'`, `H'`, `E'` are the rotated points.

In [ ]:
@udf(returnType=DoubleType())
def manhattan_dist(lat_start, lon_start, lat_end, lon_end):
    """
    The preprocessing function takes latitudes and longitudes of start position and end position,
    and returns the Manhattan distance between them.
    Args:
        lat_start: The latitude of the start position.
        lon_start: The longitude of the start position.
        lat_end: The latitude of the end position.
        lon_end: The longitude of the end position.
    Returns:
        The Manhattan distance between start and end position.
    """
    # Approximate radius of Earth in mile.
    EARTH_RADIUS = 3958.76

    # Approximate inclined degree of the streets in Manhattan.
    THETA = np.radians(-28.904)

    def haversine(pos_start, pos_end):
        """
        The helper function takes start and end position and returns the haversine distance.
        Args:
            pos_start: a latitude and a longitude of start position in np.array form.
            pos_end: a latitude and a longitude of end position in np.array form.
        Returns:
            The Haversine, the spherical distance between pos_start and pos_end on a sphere, in this case, the Earth.
        """
        rad_dist_lat = np.radians(pos_end[0] - pos_start[0])
        rad_dist_lon = np.radians(pos_end[1] - pos_start[1])
        rad_lat1 = np.radians(pos_start[0])
        rad_lat2 = np.radians(pos_end[0])
        dist = 2 * np.arcsin(
            np.sqrt(
                np.sin(rad_dist_lat / 2) ** 2
                + np.cos(rad_lat1) * np.cos(rad_lat2) * np.sin(rad_dist_lon / 2) ** 2
            )
        )
        return EARTH_RADIUS * dist

    def rotate(pos, theta):
        """
        The helper function takes position and the degree theta, and returns rotated position.
        Args:
            pos: a latitude and a longitude of position in np.array form.
            theta: the degree to rotate.
        Returns:
            Rotated position.
        """
        rotate = np.array(
            [[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]
        )
        return np.matmul(rotate, pos)

    if not (lat_start and lon_start and lat_end and lon_end):
        return -1

    # Convert positions to np.array format.
    start, end = np.array([lat_start, lon_start]), np.array([lat_end, lon_end])

    # Rotate each positions by 29' using a helper function.
    rotated_start, rotated_end = rotate(start, THETA), rotate(end, THETA)

    # Get rotated hinge point using rotated start and end point.
    rotated_hinge = np.array([rotated_start[0], rotated_end[1]])

    # Re-rotate the hinge point.
    hinge = rotate(rotated_hinge, -THETA)

    # Return the sum of the Haversine distance between start and hinge, hinge and end.
    return float(haversine(start, hinge) + haversine(hinge, end))

In [ ]:
# Calculate the Manhattan distance between "start_station" and "end_station".
bike_df = bike_df.withColumn(
    "trip_distance",
    manhattan_dist(
        col("start_station_latitude"),
        col("start_station_longitude"),
        col("end_station_latitude"),
        col("end_station_longitude"),
    ),
)

In [ ]:
# Filter out the bike_df with the same condition as the taxi_df
bike_df = bike_df.where(
    (col("trip_distance") > 0)
    & (col("trip_distance") < 20)
    & (col("trip_duration") > 0)
    & (col("start_station_latitude") > 40)
    & (col("start_station_latitude") < 41)
    & (col("end_station_latitude") > 40)
    & (col("end_station_latitude") < 41)
    & (col("start_station_longitude") > -75)
    & (col("start_station_longitude") < -73)
    & (col("end_station_longitude") > -75)
    & (col("end_station_longitude") < -73)
)

In [ ]:
bike_df.printSchema()

Check the distributions for the numerical columns. In PySpark, visualizing is expensive because the data is too large. 

For example, the NYC Taxi dataset has more than 5M rows. Therefore, approximately 3% of total data (approx. 330k rows) are extracted as a sample.

In [ ]:
%%time
taxi_sample = taxi_df.sample(0.03)
taxi_pd = taxi_sample.toPandas()

# Convert "object" type columns to the float type.
taxi_pd["trip_distance"] = taxi_pd["trip_distance"].astype(float)
taxi_pd["fare_amount"] = taxi_pd["fare_amount"].astype(float)
taxi_pd.info()

bike_sample = bike_df.sample(0.03)
bike_pd = bike_sample.toPandas()
bike_pd.info()

Since there are some extreme data points, remove them and bin the data into a box and histogram.

In [ ]:
TAXI_COLUMNS_TO_SHOW = {"trip_distance", "trip_duration", "fare_amount"}

taxi_pd_filtered = taxi_pd.query(
    "trip_distance > 0 and trip_distance < 20 \
    and trip_duration > 0 and trip_duration < 10000"
)

sns.relplot(
    data=taxi_pd_filtered,
    x="trip_distance",
    y="trip_duration",
    kind="scatter",
    ci=None,
)

for column in taxi_pd_filtered.columns:
    if column in TAXI_COLUMNS_TO_SHOW:
        _, ax = plt.subplots(1, 2, figsize=(10, 4))
        taxi_pd_filtered[column].plot(kind="box", ax=ax[0])
        taxi_pd_filtered[column].plot(kind="hist", ax=ax[1])
        plt.title(column)
        plt.show()

We found that most columns are skewed right and the trip_distance basically increases as trip_duration increases. Furthermore, most trips are done within an hour (3600 sec).

Do the similar work for the Citi Bike dataset.

In [ ]:
BIKE_COLUMNS_TO_SHOW = {"trip_distance", "trip_duration"}

bike_pd_filtered = bike_pd.query(
    "trip_distance > 0 and trip_distance < 20 \
    and trip_duration > 0 and trip_duration < 10000"
)

sns.relplot(
    data=bike_pd_filtered,
    x="trip_distance",
    y="trip_duration",
    hue="usertype",
    kind="scatter",
    ci=None,
)

The plot for trip_distance and trip_duration for Citi Bike looks different than the same plot for the Taxi dataset.

A lot of trips was longer than expected given the trip distance. The reason for this behavior seems that, unlike taxis, city bike users less likely to have an exact destination of trip, or they did not park their bicycles properly at the station. This behavior is found more on customers, who do not use the Citi Bike regularly.

To remove outliers, let's assume the average bike speed is at least 2 miles per hour, and filter them out if a trip_duration multiplied by speed is less trip_distance, and remove `Customer` user type.

In [ ]:
bike_pd_filtered = bike_pd.query(
    "trip_distance > 0 and trip_distance < 8 \
    and (trip_duration * 0.0005) <= trip_distance \
    and trip_duration > 0 and trip_duration < 3600 \
    and usertype=='Subscriber'"
)

sns.relplot(
    data=bike_pd_filtered,
    x="trip_distance",
    y="trip_duration",
    hue="usertype",
    kind="scatter",
    ci=None,
)

for column in bike_pd_filtered.columns:
    if column in BIKE_COLUMNS_TO_SHOW:
        _, ax = plt.subplots(1, 2, figsize=(10, 4))
        bike_pd_filtered[column].plot(kind="box", ax=ax[0])
        bike_pd_filtered[column].plot(kind="hist", ax=ax[1])
        plt.title(column)
        plt.show()

Based on the visualized data, apply filter to remove unrealistic values.

In [ ]:
taxi_df = taxi_df.where(
    (col("trip_duration") > 0)
    & (col("trip_duration") < 3600)
    & ((col("trip_duration") * 0.00055) <= col("trip_distance"))
)

bike_df = bike_df.where(
    (col("trip_duration") > 0)
    & (col("trip_duration") < 3600)
    & (col("usertype") == "Subscriber")
    & ((col("trip_duration") * 0.00055) <= col("trip_distance"))
)

### Feature Selection

Not all features in our dataset will be useful. 
Since the purpose of this tutorial is comparing two datasets, we need to use same features for training. There are some useful features for training the Taxi dataset, such as `fare_amount`, since Citi Bike dataset does not have it, it will not be used in this tutorial.

After choose the following columns as features, they needs to be assembled with `VectorAssembler()`, A feature transformer that merges multiple columns into a vector column.

In [ ]:
feature_cols = [
    "is_weekday",
    "start_time_in_minute",
    "start_zone_id",
    "end_zone_id",
    "trip_distance",
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Transform each column into the vector form.
taxi_transformed_data = assembler.transform(taxi_df)
bike_transformed_data = assembler.transform(bike_df)

# Split randomly with training and test data.
(taxi_training_data, taxi_test_data) = taxi_transformed_data.randomSplit([0.95, 0.05])
(bike_training_data, bike_test_data) = bike_transformed_data.randomSplit([0.95, 0.05])

### Training the Model

In PySpark, there are several Regression Models provided such as `LinearRegression`, `GeneralizedLinearRegression`, `DecisionTreeRegressor`, `RandomForestRegressor`, and `GBTRegressor`.
`GBTRegressor`, as known as `Gradient Boosted Trees` are a popular regression method using ensembles of decision trees. This algorithm iteratively trains decision trees in order to minimize a loss function.
Although this model is computationally expensive, in testing it was shown that the `GBT Regressor` provides the best results.

In [ ]:
# Define GBTRegressor
gbt = GBTRegressor(
    featuresCol="features",
    labelCol="trip_duration",
    predictionCol="pred_trip_duration",
)

# Define evaluator for r2 score.
evaluator_r2 = RegressionEvaluator(
    labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol(), metricName="r2"
)

# Define evaluator for RMSE error.
evaluator_rmse = RegressionEvaluator(
    labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol(), metricName="rmse"
)

In [ ]:
# Train a model and get predictions for the Taxi dataset. It takes several minutes.
taxi_gbt_model = gbt.fit(taxi_training_data)
taxi_gbt_predictions = taxi_gbt_model.transform(taxi_test_data)

In [ ]:
# Train a model and get predictions for the Citi Bike dataset. It takes several minutes.
bike_gbt_model = gbt.fit(bike_training_data)
bike_gbt_predictions = bike_gbt_model.transform(bike_test_data)

In [ ]:
# Evaluate the r2 score for the Taxi dataset
taxi_gbt_accuracy_r2 = evaluator_r2.evaluate(taxi_gbt_predictions)
print(f"Taxi Test GBT R2 Accuracy = {taxi_gbt_accuracy_r2}")

# Evaluate the RMSE for the Taxi dataset
taxi_gbt_accuracy_rmse = evaluator_rmse.evaluate(taxi_gbt_predictions)
print(f"Taxi Test GBT RMSE Accuracy = {taxi_gbt_accuracy_rmse}")

In [ ]:
# Evaluate the r2 score for the Citi Bike dataset
bike_gbt_accuracy_r2 = evaluator_r2.evaluate(bike_gbt_predictions)
print(f"Bike Test GBT R2 Accuracy = {bike_gbt_accuracy_r2}")

# Evaluate the RMSE for the Citi Bike dataset
bike_gbt_accuracy_rmse = evaluator_rmse.evaluate(bike_gbt_predictions)
print(f"Bike Test GBT RMSE Accuracy = {bike_gbt_accuracy_rmse}")

### Debrief the result

The trained model for the Taxi dataset shows about 66% for r2 score and 265 root-mean square error (RMSE), while the model for the Citi Boke dataset shows about 69% for r2 score and 345 RMSE. There are some restrictions to having relatively lower scores.
- Do not use CrossValidator
  - In real machine learning projects, cross-validation is an essential tool that provides us to utilize our data better. However, it re-run the training algorithm several times so it requres a lot of time and computation resource. For more information, [check this link](https://en.wikipedia.org/wiki/Cross-validation_%28statistics%29).
- Do not use every feature in the Taxi dataset.
  - The Taxi dataset has a `fare_amount` column, which has a strong correlation between the label, `trip_duration`. If that column was included, the model for the Taxi dataset shows 90+% r2 score.
  - The reason that the Taxi dataset has limited features is the purpose of this project is to compare Taxi and the Citi Bike.
- Do not use the exact coordinates for both datasets.
  - Due to privacy concerns, Taxi dataset is no longer providing the precise coordination of pickup and dropoff locations, but NYC Taxi zones in recent data.
  - Taxi Zone is a categorical number that does not represents a geographical zone defined by NYC Taxi

### Save the model to a Cloud Storage path

In [ ]:
BUCKET_FOLDER = "/tmp/bucket"

# In the testing environment, saving to GCS bucket routes to the local file system.
if os.getenv("IS_TESTING"):
    ! rm -rf $BUCKET_FOLDER
    ! mkdir $BUCKET_FOLDER
    bike_gbt_model.write().overwrite().save(f"{BUCKET_FOLDER}")
    ! gsutil cp -r $BUCKET_FOLDER $BUCKET_URI
else:
    bike_gbt_model.write().overwrite().save(f"{BUCKET_URI}/")

## Cleaning up

See [Clean up](https://cloud.google.com/vertex-ai/docs/workbench/managed/create-managed-notebooks-instance-console-quickstart#clean-up) to delete your project or the managed notebook created in this tutorial.

### Delete BigQuery dataset

In [ ]:
! bq rm -r -f $DATASET_NAME

After you delete the BigQuery dataset, you can check your Datasets in BigQuery using the following command.

In [ ]:
! bq ls

### Delete Google Cloud Storage bucket

In [ ]:
! gsutil rm -r $BUCKET_URI

### Delete Dataproc Cluster

It is not possible to delete the cluster you are currently using unless you switch the kernel to local. To delete it, you need to switch the kernel to local `Python 3` or `PySpark`, set your `CLUSTER_NAME` and `CLUSTER_REGION` manually in the following cell, and execute the `gcloud` command.

See [Deleting a cluster](https://cloud.google.com/dataproc/docs/guides/manage-cluster#console) to delete the Dataproc cluster created in this tutorial.

In [ ]:
CLUSTER_NAME = "[your-cluster-name]"
CLUSTER_REGION = "[your-cluster-region]"

In [ ]:
if not os.getenv("IS_TESTING"):
    ! gcloud dataproc clusters delete $CLUSTER_NAME --region=$CLUSTER_REGION -q